In [2]:
!pip install selenium beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.6 MB/s eta 0:00:00


In [7]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import requests
from bs4 import BeautifulSoup
import time
import json

In [8]:
HEADERS = {"User-Agent": "Mozilla/5.0"}

def save_to_json(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f, indent=2)
    print(f"Saved {len(data)} entries to {filename}")

In [13]:
from urllib.parse import urljoin

def scrape_pytorch():
    index_urls = [
        "https://pytorch.org/tutorials/intermediate/index.html",
        "https://pytorch.org/tutorials/recipes/recipes_index.html",
        "https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html"
    ]

    all_links = set()

    for index_url in index_urls:
        try:
            res = requests.get(index_url, headers=HEADERS)
            soup = BeautifulSoup(res.content, "html.parser")
            for a in soup.find_all("a", href=True):
                href = a["href"]
                if not href.endswith(".html") or "index" in href:
                    continue
                full_url = urljoin(index_url, href)
                all_links.add(full_url)
        except Exception as e:
            print(f"Error indexing {index_url}: {e}")

    print(f"Found {len(all_links)} PyTorch tutorial links")

    data = []
    for url in all_links:
        try:
            print(f"Scraping {url}")
            res = requests.get(url, headers=HEADERS)
            page = BeautifulSoup(res.content, "html.parser")
            title = page.find("h1").text.strip() if page.find("h1") else "No title"
            code_blocks = [
                pre.text.strip()
                for pre in page.find_all("pre")
                if any(kw in pre.text for kw in ["torch.", "import", "def", "class"])
            ]
            print(f" → {title}: {len(code_blocks)} code blocks")
            if code_blocks:
                data.append({
                    "url": url,
                    "title": title,
                    "code_snippets": code_blocks
                })
            time.sleep(1.2)
        except Exception as e:
            print(f"Error scraping {url}: {e}")
    return data


In [14]:
torch_data = scrape_pytorch()
save_to_json(torch_data, "pytorch_tutorials.json")

Found 175 PyTorch tutorial links
Scraping https://pytorch.org/tutorials/intermediate/realtime_rpi.html
 → Real Time Inference on Raspberry Pi 4 (30 fps!)¶: 9 code blocks
Scraping https://pytorch.org/tutorials/intermediate/process_group_cpp_extension_tutorial.html
 → Customize Process Group Backends Using Cpp Extensions¶: 5 code blocks
Scraping https://pytorch.org/tutorials/recipes/torch_logs.html
 → (beta) Using TORCH_LOGS python API with torch.compile¶: 3 code blocks
Scraping https://pytorch.org/tutorials/beginner/nn_tutorial.html
 → What is torch.nn really?¶: 31 code blocks
Scraping https://pytorch.org/tutorials/recipes/deployment_with_flask.html
 → Deploying with Flask¶: 7 code blocks
Scraping https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html
 → Saving And Loading A General Checkpoint¶: 0 code blocks
Scraping https://pytorch.org/tutorials/advanced/torch_script_custom_ops.html
 → Extending TorchScript with Custom C++ Operators¶: 18 code blocks